# 2 つの文字列 $X, Y$ のシャッフル


「パタトカクシーー」のような文字列を、「パトカー」と「タクシー」のシャッフルと呼ぶことにする。


3 つの文字列 $A[1..n], B[1..m], C[1..n+m]$ が与えられた時、$C[2..n+m]$は$A[2..n], B[1..m]$または$A[1..n], B[2..m]$のシャッフルである。

A の最初の i 文字と B の最初の j 文字から C の最初の i+j 文字が構成できるかを、関数 `isShuffle(i, j)`で判定する。関数は次の再帰方程式で定義する。なお、定義にあたってインデックスを 0 オリジンに変えている。

$$
\text{isShuffle}(i, j) =
\begin{cases}
\text{True} & \text{if } i = 0 \text{ and } j = 0 \\
(\text{isShuffle}(i-1, j) \land A[i] = C[i+j]) \lor (\text{isShuffle}(i, j-1) \land B[j] = C[i+j]) & \text{otherwise}
\end{cases}
$$


In [7]:
%env PYTHON_LOGLEVEL=DEBUG

env: PYTHON_LOGLEVEL=DEBUG


In [8]:
from logging import getLogger, StreamHandler, WARNING
import os

PYTHON_LOGLEVEL = os.environ.get("PYTHON_LOGLEVEL", WARNING)
logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(PYTHON_LOGLEVEL)
logger.setLevel(PYTHON_LOGLEVEL)
logger.addHandler(handler)
logger.propagate = False

In [37]:
from collections import defaultdict


def wrap(a: str, b: str, c: str) -> bool:
    if len(a) + len(b) != len(c):
        return False

    mem = defaultdict(lambda: defaultdict(lambda: None))
    mem[0][0] = True

    # Take the last index as arguments
    def isShuffle(i, j) -> bool:
        if mem[i][j] is not None:
            logger.debug(f"{i=}, {j=}, {mem[i][j]=}")
            return mem[i][j]

        if i < 0 and j < 0:
            return True

        lastCharFromA = i >= 0 and isShuffle(i - 1, j) and c[i + j + 1] == a[i]
        lastCharFromB = j >= 0 and isShuffle(i, j - 1) and c[i + j + 1] == b[j]
        logger.debug(
            f"{i=}, {j=}, {a[0:i+1]=}, {b[0:j+1]=}, {c[0:i+j+2]=}, {lastCharFromA=}, {lastCharFromB=}"
        )
        mem[i][j] = lastCharFromA or lastCharFromB
        return mem[i][j]

    return isShuffle(len(a) - 1, len(b) - 1)

In [38]:
expected = True
actual = wrap("パトカー", "タクシー", "パタトカクシーー")
assert expected == actual

i=-1, j=0, a[0:i+1]='', b[0:j+1]='タ', c[0:i+j+2]='パ', lastCharFromA=False, lastCharFromB=False
i=-1, j=1, a[0:i+1]='', b[0:j+1]='タク', c[0:i+j+2]='パタ', lastCharFromA=False, lastCharFromB=False
i=-1, j=2, a[0:i+1]='', b[0:j+1]='タクシ', c[0:i+j+2]='パタト', lastCharFromA=False, lastCharFromB=False
i=-1, j=3, a[0:i+1]='', b[0:j+1]='タクシー', c[0:i+j+2]='パタトカ', lastCharFromA=False, lastCharFromB=False
i=-1, j=2, mem[i][j]=False
i=-1, j=1, mem[i][j]=False
i=0, j=0, mem[i][j]=True
i=0, j=1, a[0:i+1]='パ', b[0:j+1]='タク', c[0:i+j+2]='パタト', lastCharFromA=False, lastCharFromB=False
i=0, j=2, a[0:i+1]='パ', b[0:j+1]='タクシ', c[0:i+j+2]='パタトカ', lastCharFromA=False, lastCharFromB=False
i=0, j=3, a[0:i+1]='パ', b[0:j+1]='タクシー', c[0:i+j+2]='パタトカク', lastCharFromA=False, lastCharFromB=False
i=0, j=2, mem[i][j]=False
i=0, j=1, mem[i][j]=False
i=0, j=0, mem[i][j]=True
i=0, j=-1, a[0:i+1]='パ', b[0:j+1]='', c[0:i+j+2]='パ', lastCharFromA=True, lastCharFromB=False
i=1, j=-1, a[0:i+1]='パト', b[0:j+1]='', c[0:i+j+2]='パタ', la